## Install Packages

In [1]:
!conda install -y -c conda-forge faiss-gpu
!apt-get -y update
!apt-get -y install libatlas-base-dev

done
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - faiss-gpu


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.14.0               |   py37h89c1867_0        1010 KB  conda-forge
    toolz-0.12.1               |     pyhd8ed1ab_0          51 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.0 MB

The following NEW packages will be INSTALLED:

  toolz              conda-forge/noarch::toolz-0.12.1-pyhd8ed1ab_0

The following packages will be UPDATED:

  conda                               4.12.0-py37h89c1867_0 --> 4.14.0-py37h89c1867_0



toolz-0.12.1         | 51 KB     | ##################################### | 100% 
conda-4.14.0         | 1010 KB   | ##################################### | 100% 
Preparing transaction: done
Ver

## Load Data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.random_projection import GaussianRandomProjection

from tqdm import tqdm

import faiss

In [3]:
df = pd.read_csv("story_dataset.csv")
df

,prompt_id,prompt,story,hidden_state_file,len_generated_story,len_new_story
0,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Blaz...,./hidden_states/prompt_1.npz,270,271
1,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Spar...,./hidden_states/prompt_1.npz,349,350
2,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Scor...,./hidden_states/prompt_1.npz,278,278
3,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,117,118
4,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,129,130
...,...,...,...,...,...,...
9995,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,289,290
9996,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,119,119
9997,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,127,128
9998,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,441,441


In [4]:
max_story_len = max(df["len_generated_story"])
max_story_len

522

In [5]:
hidden_states_by_layer = {}
NUM_PROMPTS = 10

for prompt_id in range(1, NUM_PROMPTS + 1):
    with np.load(f'./hidden_states/prompt_{prompt_id}.npz') as loaded_data:
        for i in tqdm(range(1000)):
            curr_hidden_states = loaded_data[f"arr_{i}"][0]
#             print(curr_hidden_states.shape)
            for layer in range(3, 4):
                padded_arr = np.zeros((max_story_len, 512))
                padded_arr_len = len(curr_hidden_states[layer][0])
                
                padded_arr[:padded_arr_len] = curr_hidden_states[layer][0]
                
                padded_arr = padded_arr.flatten().astype('float32') #FAISS expects data in type float32 instead of float64 - saves memory too!
#                 print(padded_arr.shape)
                
                if(f"layer_{layer}" in hidden_states_by_layer):
                    hidden_states_by_layer[f"layer_{layer}"].append(padded_arr)
                else:
                    hidden_states_by_layer[f"layer_{layer}"] = [padded_arr]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:37<00:00, 26.97it/s]


In [6]:
layer_hs_array = np.array(hidden_states_by_layer["layer_3"])
layer_hs_array.shape

(10000, 267264)

## Layer 3 Clustering

In [7]:
# Use original vectors for clustering - uncomment next line and comment out last two lines

# dim_reduced_vecs = layer_hs_array

random_projector = GaussianRandomProjection(random_state = 42)
dim_reduced_vecs = random_projector.fit_transform(layer_hs_array).astype('float32')

In [8]:
dim_reduced_vecs = np.array([v / np.linalg.norm(v) for v in dim_reduced_vecs])
dim_reduced_vecs.shape

(10000, 7894)

In [9]:
# K-means Clustering

ncentroids = NUM_PROMPTS
niter = 20
verbose = True
dim = dim_reduced_vecs.shape[1]
kmeans = faiss.Kmeans(dim, ncentroids, niter = niter, verbose = verbose, gpu = True, nredo = 10, spherical = True, max_points_per_centroid = 1000)
kmeans.train(dim_reduced_vecs)

Clustering 10000 points in 7894D to 10 clusters, redo 10 times, 20 iterations
  Preprocessing in 0.07 s
Outer iteration 0 / 10
  Iteration 19 (1.93 s, search 1.23 s): objective=4940.35 imbalance=1.230 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 10
  Iteration 19 (3.83 s, search 2.45 s): objective=4926.19 imbalance=1.993 nsplit=0       
Outer iteration 2 / 10
  Iteration 19 (5.77 s, search 3.67 s): objective=4956.88 imbalance=1.084 nsplit=0       
Objective improved: keep new clusters
Outer iteration 3 / 10
  Iteration 19 (7.72 s, search 4.90 s): objective=4948.6 imbalance=1.029 nsplit=0        
Outer iteration 4 / 10
  Iteration 19 (9.67 s, search 6.12 s): objective=4948.52 imbalance=1.093 nsplit=0       
Outer iteration 5 / 10
  Iteration 19 (11.61 s, search 7.35 s): objective=4944.27 imbalance=1.158 nsplit=0       
Outer iteration 6 / 10
  Iteration 19 (13.53 s, search 8.57 s): objective=4933.53 imbalance=1.555 nsplit=0       
Outer iteration 7 / 10
  It

4956.8818359375

In [10]:
kmeans.centroids #cluster centers

array([[ 0.02099111, -0.01000931,  0.0094079 , ..., -0.00449789,
         0.00683649,  0.01301471],
       [ 0.02386447, -0.00512525,  0.01142995, ..., -0.00642765,
         0.00806163,  0.01200529],
       [ 0.01048543, -0.01008676,  0.00876845, ..., -0.00099966,
         0.00912536,  0.01486209],
       ...,
       [ 0.01664427, -0.00984779,  0.00937519, ..., -0.00604355,
         0.00819936,  0.00902302],
       [ 0.01459649, -0.00397317,  0.00901716, ..., -0.00094649,
         0.01285713,  0.01601537],
       [ 0.01237294, -0.00952258,  0.00877464, ..., -0.00475856,
         0.01350583,  0.01303124]], dtype=float32)

In [11]:
for centroid in kmeans.centroids:
    print(np.linalg.norm(centroid))

1.0
1.0
1.0000001
0.99999994
1.0000001
1.0000002
0.99999994
1.0000005
0.9999999
1.0000001


In [12]:
kmeans.obj #inertia at each iteration

array([3079.28076172, 4782.8515625 , 4853.50439453, 4887.18945312,
       4904.16894531, 4914.97070312, 4922.67822266, 4927.06835938,
       4930.015625  , 4932.55322266, 4934.87548828, 4936.60009766,
       4937.97070312, 4938.89404297, 4939.47167969, 4939.79150391,
       4940.04052734, 4940.17285156, 4940.27539062, 4940.35400391,
       3249.52050781, 4777.4609375 , 4824.40917969, 4869.34277344,
       4899.15820312, 4906.80615234, 4909.93701172, 4911.77978516,
       4912.90869141, 4914.20263672, 4916.12060547, 4919.015625  ,
       4922.33251953, 4924.10644531, 4924.69628906, 4924.99365234,
       4925.26220703, 4925.56835938, 4925.93310547, 4926.19140625,
       3041.2097168 , 4777.65625   , 4854.98925781, 4895.93164062,
       4910.78417969, 4920.24658203, 4926.42382812, 4931.77636719,
       4936.97412109, 4941.95458984, 4945.78515625, 4948.13232422,
       4950.28369141, 4952.12939453, 4953.74072266, 4954.91503906,
       4955.6015625 , 4956.08398438, 4956.54394531, 4956.88183

In [13]:
pd.Series(kmeans.index.search(dim_reduced_vecs.astype(np.float32), 1)[1].flatten()).value_counts()

0    1309
4    1276
9    1219
8    1160
7    1120
2    1106
5    1071
6     744
1     569
3     426
dtype: int64

In [14]:
normalized_vecs = [v / np.linalg.norm(v) for v in dim_reduced_vecs]

In [15]:
cos_similarities = normalized_vecs @ kmeans.centroids.T
classifications = np.argmax(cos_similarities, axis=1)

In [16]:
pd.Series(classifications).value_counts()

0    1309
4    1276
9    1219
8    1160
7    1120
2    1106
5    1071
6     744
1     569
3     426
dtype: int64